In [ ]:
import chipwhisperer as cw
scope = cw.scope(prog_speed=5e6)

In [ ]:
TARGET_FREQ = 25e6
#TARGET_FREQ = 100e6
STREAM = True # TARGET_FREQ must be max 25 MHz to use streaming
if STREAM:
    assert TARGET_FREQ <= 25e6

In [ ]:
from chipwhisperer.capture.targets.CW310 import CW310
target = cw.target(scope, CW310, bsfile=None)

target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(False, 2)
target.pll.pll_outfreq_set(TARGET_FREQ, 1)

sstarget = cw.target(scope)
sstarget.baud = int(115200 * TARGET_FREQ/100e6)

In [ ]:
force_program = False
if not(target.fpga.isFPGAProgrammed() or force_program):
    bsfile = '../../../../Downloads/lowrisc_systems_chip_earlgrey_cw310_0.1.bit'
    status = target.fpga.FPGAProgram(open(bsfile, "rb"), exceptOnDoneFailure=False, prog_speed=10e6)
    assert status, "FPGA Done pin failed to go high"

In [ ]:
import sys
sys.path.append("../../..")
from opentitan.otbn_ecc_hacky.util.fpga.cw_spiflash import *

In [ ]:
firmware = '../../../../Downloads/otbn_ecdsa_p256_test_fpga_cw310.bin'
prog = SPIProgrammer(firmware, "CW310")
#prog.run(target) # this is run during the capture

In [ ]:
# keep default setup:
scope.io.tio1 = 'serial_tx'
scope.io.tio2 = 'serial_rx'

In [ ]:
if scope._is_husky:
    scope.clock.clkgen_src = 'extclk'
    scope.clock.clkgen_freq = TARGET_FREQ
    scope.clock.adc_mul = 1
    # we'll be playing with the clock so we don't want to get errors when Husky notices the clock changing:
    scope.clock.extclk_monitor_enabled = False
    scope.gain.db = 13
else:
    scope.clock.adc_src = "extclk_x1"
    scope.gain.db = 13 # TODO- this may not be the best value

In [ ]:
# these are specific to Husky!
if STREAM:
    scope.adc.stream_mode = True
    scope.adc.bits_per_sample = 8
    scope.adc.samples = 8000000
    scope.adc.decimate = 1
    scope.adc._timeout = 5
    scope.gain.db = 22
else:
    scope.adc.stream_mode = False
    scope.adc.bits_per_sample = 12
    scope.adc.samples = 131070
    scope.adc.decimate = 10
    scope.adc._timeout = 5
    scope.gain.db = 13

In [ ]:
def num_to_bytearray(num, bit_length=256):
    return bytearray([num >> (8*i) & 0xFF for i in range((bit_length - 1) // 8,-1,-1)][::-1])

In [ ]:
# capture loop:
N = 4
reset_firmware = True
all_cycles = []
waves = []
from tqdm.notebook import trange

for i in trange(N):
    if reset_firmware:
        if TARGET_FREQ != 100e6:
            target.pll.pll_outfreq_set(100e6, 1)
        prog.run(target)
        if TARGET_FREQ != 100e6:
            target.pll.pll_outfreq_set(TARGET_FREQ, 1)
        time.sleep(0.3)
        sstarget.simpleserial_write('v', bytearray())
        
    scope.arm()
    if scope._is_husky:
        start_cycles = 0
    else:
        start_cycles = scope.adc.trig_count

    if i < 2:
        k = num_to_bytearray(0x0f0f0fffffffffffffffffffffffffff00000000000000000000000000000000)
    else:
        k = num_to_bytearray(0xffffffffffffffffffffffffffffffff00000000000000000000000000000000)

    sstarget.simpleserial_write('p', k)
    ret = scope.capture()
    if ret:
        print('Warning: timeout during capture')

    cycles = scope.adc.trig_count - start_cycles
    all_cycles.append(cycles)
    if cycles != 7534831:
        print("Observed odd number of cycles: %d" % cycles)
    waves.append(scope.get_last_trace())


In [ ]:
all_cycles

In [ ]:
scope.adc.errors

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook

output_notebook(INLINE)

In [ ]:
p = figure(plot_width=2000)

samples = 300000
base = 0
#samples = len(waves[0])
xrange = range(samples)
#p.line(xrange, waves[3][base:base+samples], line_color="red")
p.line(xrange, waves[0][base:base+samples], line_color="green")
#p.line(xrange, waves[2][base:base+samples], line_color="blue")
#p.line(xrange, waves[3][base:base+samples], line_color="orange")

In [ ]:
show(p)

In [ ]:
d = figure(plot_width=2000)

samples = 300000
base = 0
#samples = len(waves[0])
xrange = range(samples)
d.line(xrange, waves[0][base:base+samples] - waves[3][base:base+samples], line_color="red")
#d.line(xrange, waves[3][base:base+samples] - waves[2][base:base+samples], line_color="blue")

In [ ]:
show(d)